In [7]:
import pandas as pd
from konlpy.tag import Okt
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# OKT 형태소 분석기 로드
okt = Okt()

# KoBERT 모델 및 토크나이저 로드 (KLUE BERT로 변경 가능)
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = BertModel.from_pretrained('klue/bert-base')

# 입력 텍스트를 BERT 입력으로 변환하고 임베딩을 생성하는 함수
def get_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

# OKT를 사용해 텍스트를 원형으로 변환하는 함수
def normalize_text(text):
    morphs = okt.morphs(text, stem=True)  # OKT를 사용해 원형으로 변환
    return ' '.join(morphs)

# 코사인 유사도 계산 함수
def cosine_similarity_review_keywords(review, keywords):
    review_emb = get_embedding(review, tokenizer, model)
    keywords_emb = get_embedding(keywords, tokenizer, model)
    return cosine_similarity(review_emb, keywords_emb)[0][0]

C:\Users\HWAN\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# CSV 파일 불러오기
df = pd.read_csv('saperate_reviews_tags/'+'3977488_keywords.csv')

# NaN 값 제거
df = df.dropna(subset=['Review', 'common_keywords_3_plus'])

# 결과 저장을 위한 리스트
similarity_scores = []

# 리뷰와 키워드 간 유사도 계산
for idx, row in df.iterrows():
    review = normalize_text(row['Review'])  # 리뷰를 원형으로 변환
    keywords = normalize_text(row['common_keywords_3_plus'])  # 키워드도 원형으로 변환
    
    # 리뷰와 키워드 간 코사인 유사도 계산
    similarity = cosine_similarity_review_keywords(review, keywords)
    similarity_scores.append(similarity)

# 유사도 결과를 데이터프레임으로 저장
df['similarity_score'] = similarity_scores

# 유사도 결과 출력
df[['Review', 'common_keywords_3_plus', 'similarity_score']]

,Review,common_keywords_3_plus,similarity_score
0,글로니 말 모 말 모 글로니 핏 이 쁜 거 두말하면 입 아파요 허리 얇고 긴 체형 ...,"가슴, 개, 넓다, 덜하다, 듬직하다, 똥꼬, 말, 숨기다, 어깨, 치마, 태평양, 허리",0.593610
1,예약 배송이라 쇼룸에서 사이즈 입어보고 샀는데 좀 다른 거 같아 요 배송된 게 좀 ...,"민소매, 배송, 부담스럽다, 불편하다, 소재, 쇼룸, 수영복, 예약, 판도",0.477697
2,화사해 보이고 예쁘지만 기장이 조금만 더 길었느면 좋겠음 많이 짧아서 좀 부담스러울 수도,"기장, 길었느, 보이, 부담스럽다, 수도, 짧다, 화사하다",0.658941
3,두 겹이라 비치 지도 않고 완전 합니다 올해 원피스 중 제일 이쁜듯 요 너무 맘에 ...,"겹, 들어서다, 비치, 올해, 원피스",0.557106
4,재질이 해서 흰색임에도 비침이 거의 없어서 좋았어요 노란 기나 푸른기 없이 깨끗한 ...,"기본, 깨끗하다, 디자인, 사이즈, 신경, 아우터, 재질, 정석, 흰색",0.458985
...,...,...,...
88,작은 제 키에는 살짝 긴 감이 있지만 무척 예쁘고 편합니다,"무척, 작다, 키, 편하다",0.758003
89,딱 붙는 핏 찾았었는데 생각한 대로에요 여기저기 입 기 좋아요,"대로, 붙다, 여기저기, 찾다",0.569270
90,믿고 사는 글로니 핏 너무 예뻐요 완전 깔끔해요 다른 색도 살거에요,"깔끔하다, 믿다, 살다, 색도",0.626242
91,블랙도 사고 싶을 정도 류 예쁜데 너무 짧음 소재는 너무 얇지 않아 좋아요,"블랙, 사고, 소재, 얇다, 짧다",0.608817


In [ ]:
# 유사도 결과를 파일로 저장 (선택 사항)
df.to_csv('similarity_results.csv', index=False)